In [ ]:
# 이미지 스플릿 하기
# https://inhovation97.tistory.com/36

import os
import glob
import math
import shutil
import random

# 원본 이미지 폴더속 이미지 개수
doc_img = sorted(glob.glob('D:/coding/data_set/lomin/train_copy/train/'+'/*'))
print('doc_img 폴더 이미지 개수 : {}'.format(len(doc_img)))

# 나눌 이미지 수
doc_split = round(len(doc_img)*0.05)
print('doc_split : split 이미지 수 : {}'.format(doc_split))

# 이미지 나눌 모듈
def split(img_list, test_count, train_path, test_path):

    test_files=[]
    for i in random.sample( img_list, test_count ):
        test_files.append(i)
    
    # 차집합으로 train/test 리스트 생성하기
    train_files = [x for x in img_list if x not in test_files]

    for k in train_files:
        shutil.copy(k, train_path)

    for c in test_files:
        shutil.copy(c, test_path)

    print('train 폴더 이미지 개수 : {}\n test 폴더 이미지 개수 : {}'.format(len(glob.glob(train_path+'/*')), len(glob.glob(test_path+'/*'))))

doc_img 폴더 이미지 개수 : 9558
doc_split : split 이미지 수 : 478


In [ ]:
# 이미지 나누기 (각 지정 폴더로 들어감)
tr_path = 'D:/coding/data_set/lomin/temp_test/train'
te_path = 'D:/coding/data_set/lomin/temp_test/test'

split(doc_img, doc_split, tr_path, te_path)

train 폴더 이미지 개수 : 9080
 test 폴더 이미지 개수 : 478


---

In [2]:
# 이미지 여백만들고 회전시키기 원본

import os
import glob
import cv2

test_path = 'D:/coding/data_set/lomin/temp_test/test' # test path

img_list = sorted(glob.glob(test_path + '/*'))

for i in img_list:
    # print(i.split('\\')[1].split('.')[0])
    src = cv2.imread(i, cv2.IMREAD_COLOR)
    height, width, channel = src.shape
    my_angle = 0

    ##                                  중심점(center), 각도(angle), 비율(scale)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
    # 비율 : 가로길이 / 대각선

    dst = cv2.warpAffine(src, matrix, (width, height))

    # 변경 이미지 저장 (파일명_각도)
    cv2.imwrite('D:/coding/data_set/lomin/temp_test/test_after/' + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)



In [ ]:
import os
import glob
import cv2

test_path = 'D:/coding/data_set/lomin/temp_test/test' # test path

img_list = sorted(glob.glob(test_path + '/*'))

# 설정 각도에 따라 저장.
def lotation_img(my_angle):
    for i in img_list:
        # print(i.split('\\')[1].split('.')[0])
        src = cv2.imread(i, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        

        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
        # 비율 : 가로길이 / 대각선

        dst = cv2.warpAffine(src, matrix, (width, height))

        # 변경 이미지 저장 (파일명_각도)
        cv2.imwrite('D:/coding/data_set/lomin/temp_test/test_after/' + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)

In [ ]:
lotation_img(0)